In [4]:
import gensim

print(gensim.__version__)

# Modell mit Top 5000 deutschen Worten laden
model = gensim.models.KeyedVectors.load_word2vec_format("min5000.model", binary=True)

3.6.0


In [18]:
matches = model.most_similar(positive=["Muenchen"], negative=None, topn=10)

matches

C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Muenchner', 0.7908819913864136),
 ('Augsburg', 0.7286888360977173),
 ('Nuernberg', 0.7239000797271729),
 ('FC_Bayern', 0.7078144550323486),
 ('Dortmund', 0.7041417360305786),
 ('Bayern', 0.6823953986167908),
 ('Hamburg', 0.6818069219589233),
 ('Duesseldorf', 0.6732111573219299),
 ('Stuttgart', 0.6708148717880249),
 ('Regensburg', 0.6612198352813721)]

In [16]:
# Muenchen zu Bayern
# Hannover zu ???

matches = model.most_similar(positive=["Hannover", "Bayern"], negative=["Muenchen"], topn=10)

matches

C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Niedersachsen', 0.7528911828994751),
 ('Hessen', 0.6565377712249756),
 ('Braunschweig', 0.6402816772460938),
 ('Bremen', 0.6086529493331909),
 ('Schleswig-Holstein', 0.5892189741134644),
 ('Sachsen', 0.585659384727478),
 ('Baden-Wuerttemberg', 0.567304253578186),
 ('Nordrhein-Westfalen', 0.5669426918029785),
 ('Sachsen-Anhalt', 0.5653688907623291),
 ('Wolfsburg', 0.5581339001655579)]